# Batch download all documents

for LOVT VISION and LOVT VIBE

## Instantiate Alasco client

In [1]:
from Alasco.alasco import Alasco
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import os
import re

_ = load_dotenv(find_dotenv(raise_error_if_not_found=True))
token = os.environ["token_hines"]
key = os.environ["key_hines"]
alasco = Alasco(token=token, key=key, verbose=True)

outputs/2024-08-21 was created


## Fetch data for these projects

In [2]:
dfs = alasco.data_fetcher.get_all_df(property_name="LOVT")

## Transform Datasets

In [7]:
df_core = alasco.data_transformer.consolidate_core_DataFrames(dfs=dfs)
df_contracts = df_core.copy()
df_invoices = alasco.data_transformer.consolidate_invoices_DataFrame(df_core=df_core, df_invoices=dfs["invoices"])
df_change_orders = alasco.data_transformer.consolidate_change_orders_DataFrame(df_core=df_core, df_change_orders = dfs["change_orders"])


## Download documents

In [20]:
df_core.project_name.unique()

array(['51104 - 002 LOVT VISION', '51102 - 001 LOVT VIBE'], dtype=object)

In [9]:
project_names = ["51104 - 002 LOVT VISION", "51102 - 001 LOVT VIBE"]
core_path = f"outputs/{alaco.document_downloader.today}"

for project_name in project_names:
    subset_df_core = df_core[df_core["project_name"] == project_name].copy()
    subset_df_contracts = df_contracts[df_contracts["project_name"] == project_name].copy()
    subset_df_invoices = df_invoices[df_invoices["project_name"] == project_name].copy()
    subset_df_change_orders = df_change_orders[df_change_orders["project_name"] == project_name].copy()

    # Set path for downloading documents for this project
    alasco.document_downloader.download_path = core_path + "/LOVT" + "/" + project_name

    # Download documents
    alasco.document_downloader.download_contracts(df=subset_df_contracts)
    alasco.document_downloader.download_change_orders(df=subset_df_change_orders)
    alasco.document_downloader.download_invoices(df=subset_df_invoices)



Created directory: outputs/2024-08-21/LOVT/51104 - 002 LOVT VISION/LOVT/51104 - 002 LOVT VISION/contracts
Created directory: outputs/2024-08-21/LOVT/51104 - 002 LOVT VISION/LOVT/51104 - 002 LOVT VISION/change_orders
Created directory: outputs/2024-08-21/LOVT/51104 - 002 LOVT VISION/LOVT/51104 - 002 LOVT VISION/invoices


In [ ]:
# alasco.document_downloader.download_contracts(df=df_core.iloc[:10,:], sub_folder="contracts")
# alasco.document_downloader.download_invoices(df=df_invoices.iloc[:10,:], sub_folder="invoices")
# alasco.document_downloader.download_change_orders(df=df_change_orders.iloc[:10,:], sub_folder = "change_orders")

In [21]:
subset_df_change_orders.head()

,property_id,property_name,project_id,project_name,contract_unit_id,contract_unit_name,contract_id,contract_name,contract_number,contractor_id,contractor_name,change_order_id,change_order_name,change_order_identifier
0,a6fe95ce-01d9-4fb7-a36f-9c6632110099,06-LOVT,053b0bc8-e945-49a0-88ed-98c1ca497cf6,51104 - 002 LOVT VISION,9c768256-7019-4de3-aa04-ef89a5f9feda,3-551-1 Technische Aussenanlagen - Abwasseranl...,b3e104e5-fc5c-4c54-9486-56c8cfa8e208,51104-002-3-551-1-0219-Sickerschacht-Heinrich ...,51104-002-3-551-1-0219,97e21858-34ac-4822-8d85-48753ff04a1c,Heinrich Preis GmbH (1074998),6fde9c97-b471-4a9e-93ba-8fa404681234,Miete Bauzaun,01
1,a6fe95ce-01d9-4fb7-a36f-9c6632110099,06-LOVT,053b0bc8-e945-49a0-88ed-98c1ca497cf6,51104 - 002 LOVT VISION,9c768256-7019-4de3-aa04-ef89a5f9feda,3-551-1 Technische Aussenanlagen - Abwasseranl...,b3e104e5-fc5c-4c54-9486-56c8cfa8e208,51104-002-3-551-1-0219-Sickerschacht-Heinrich ...,51104-002-3-551-1-0219,97e21858-34ac-4822-8d85-48753ff04a1c,Heinrich Preis GmbH (1074998),6fde9c97-b471-4a9e-93ba-8fa404681234,Miete Bauzaun,01
2,a6fe95ce-01d9-4fb7-a36f-9c6632110099,06-LOVT,053b0bc8-e945-49a0-88ed-98c1ca497cf6,51104 - 002 LOVT VISION,b247c644-d091-4e8f-90cf-87aa10fff0e7,3-391-8 Provisorische Baukonstruktionen,fd23d931-3a7d-4ce4-ba1b-11b8a23a8a6a,51104-002-3-391-8-0418_Deda_Absturzsicherung 1500,51104-002-3-391-8-0418,c8acee0e-1831-4462-b3bd-12717af19be2,Deda Bau GmbH (1103584),9351102e-4af8-465e-9821-c86dcde42c45,NA01 Nachtragsangebot für zusätzliche provisor...,01
3,a6fe95ce-01d9-4fb7-a36f-9c6632110099,06-LOVT,053b0bc8-e945-49a0-88ed-98c1ca497cf6,51104 - 002 LOVT VISION,2c670f63-4d94-4a61-af03-8445e62cc694,3-391-7 Gerüste,317a6514-f334-435f-ba57-a41e40fd1cec,51104-002-3-391-7-0222-Gerüst Treppenturm Ster...,51104-002-3-391-7-0222,abc455b6-5f64-481c-987b-42ea4931485e,Wieser Malerbetrieb Gerüstbau GmbH (1091399),8ca31d1d-c7d6-490c-b009-976cf0d4e3e9,Erweiterung Schutzdach DG,03
4,a6fe95ce-01d9-4fb7-a36f-9c6632110099,06-LOVT,053b0bc8-e945-49a0-88ed-98c1ca497cf6,51104 - 002 LOVT VISION,2c670f63-4d94-4a61-af03-8445e62cc694,3-391-7 Gerüste,317a6514-f334-435f-ba57-a41e40fd1cec,51104-002-3-391-7-0222-Gerüst Treppenturm Ster...,51104-002-3-391-7-0222,abc455b6-5f64-481c-987b-42ea4931485e,Wieser Malerbetrieb Gerüstbau GmbH (1091399),4350ea84-60dc-4a83-8819-9742718fe58f,Treppenlauf Lichtgraben,04


In [17]:
client = Alasco.Alasco()

AttributeError: module 'Alasco' has no attribute 'Alasco'